In [1]:
import os 

import fiona #must be import before geopandas
import geopandas as gpd
import rasterio
#import rioxarray as xr
import rioxarray

import re
import rtree
import shapely
import pickle

#from cartopy import crs
import collections
import cv2
import math
from glob import glob
from tqdm.notebook import tqdm_notebook
import tqdm
# Standard packages
import tempfile
import warnings
import urllib
import shutil

# Less standard, but still pip- or conda-installable
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

import data_eng.az_proc as ap
import data_eng.form_calcs as fc

from lxml.etree import Element,SubElement,tostring
import xml.dom.minidom
from xml.dom.minidom import parseString
import xml.etree.ElementTree as et
from xml.dom import minidom

#import requests
from PIL import Image
from io import BytesIO
import tqdm
from skimage.metrics import structural_similarity as compare_ssim
import imutils

import psutil

## File Paths

In [2]:
parent_directory = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//"

tile_names_tile_urls_complete_array = np.load("image_download_azure/tile_name_tile_url_complete_array.npy")

tiles_labeled = "tile_name_tile_url_labeled.npy"
tiles_labeled_from_complete_set = np.load(tiles_labeled)

tracker_file_path = 'outputs/tile_img_annotation_annotator.npy'
tile_img_annotation = np.load(tracker_file_path)
tile_img_annotation_annotator = np.load("outputs/tile_img_annotation_annotator.npy")

tiles_errors = 'tile_name_tile_url_error_downloading.npy'
tiles_errors = np.load(tiles_errors)

#create folder to hold tiles in completed dataset

tiles_complete_dataset_path = os.path.join(parent_directory,"complete_dataset","tiles")
tiles_xml_complete_dataset_path = os.path.join(parent_directory,"complete_dataset","tiles_xml")
os.makedirs(tiles_complete_dataset_path, exist_ok=True)

tile_names_tile_urls_complete_array_unique_standard_tile_names = np.load("tile_names_tile_urls_complete_array_unique_standard_tile_names.npy")
image_characteristics = pd.read_csv("image_characteristics.csv")
tile_names_tile_urls_complete_array_unique_standard_tile_names = np.load("tile_names_tile_urls_complete_array_unique_standard_tile_names.npy")

#NAIP quad map
quad_indicies_path = "C:/Users/rapiduser/Box/EPA STAR 2019 (Community Resistance to Environmental Disasters)/Data/AST Datasets/MapIndices_National_GDB/MapIndices_National_GDB.gdb"
#fiona.listlayers(quad_indicies_path)

## File Paths

In [ ]:
###CONFIRM CHIP X AND Y INDEXES

In [3]:
def determine_SE_NW_lat_lon(tile_path, tile_name):
    ## Get tile locations
    da = rioxarray.open_rasterio(os.path.join(tile_path, tile_name)) ## Read the data
    # Compute the lon/lat coordinates with rasterio.warp.transform
    da = da.rio.reproject("EPSG:4326") #reproject
    lons, lats = np.meshgrid(da['x'], da['y'])
    return(lons, lats)

In [4]:
da = rioxarray.open_rasterio(os.path.join(tiles_complete_dataset_path, tile_name+".tif")).rio.reproject("EPSG:4326") ## Read the data
    # Compute the lon/lat coordinates with rasterio.warp.transform
da = da.rio.reproject("EPSG:4326") #reproject
#    lons, lats = np.meshgrid(da['x'], da['y'],dtype='uint8')
#    return(lons, lats)

NameError: name 'tile_name' is not defined

In [8]:
def image_characteristics(images_and_xmls_by_tile_path, tiles_dir):#, verified_positive_jpgs):
    """
    Only characterisizes images for which the corresponding tile is downloaded
    Args:
    images_and_xmls_by_tile_path(str): path to directory containing folders (named by tiles); where each folder contains the images/xmls
    tiles_dir(str): path to the directory containing tiles
    
    Returns:
    image_characteristics(pandadataframe):containing image characterisitcs 
    """ 
    state = []
    resolution = []
    year = []
    capture_date  = []
    utm_zone  = []
    standard_tile_names = []
    chip_names = []
    NW_coordinates_pixel = []
    SE_coordinates_pixel = []
    NW_coordinates_lonlat = []
    SE_coordinates_lonlat = []
    row_indicies = []
    col_indicies = []
    full_path  = []
    root = []
    
    item_dim = int(512)
    folders_of_images_xmls_by_tile = os.listdir(images_and_xmls_by_tile_path)
    for tile_name in folders_of_images_xmls_by_tile:
        #specify image/xml paths for each tile
        positive_image_dir = os.path.join(images_and_xmls_by_tile_path, tile_name, "chips_positive")
        positive_xml_dir = os.path.join(images_and_xmls_by_tile_path, tile_name, "chips_positive_xml")
        #load a list of images/xmls for each tile
        positive_images = os.listdir(positive_image_dir)
        positive_xmls = os.listdir(positive_xml_dir)
        
        #read in tile
        tile = cv2.imread(os.path.join(tiles_dir, tile_name + ".tif"))
        #determine the lat/lon for each tile 
        lons, lats = determine_SE_NW_lat_lon(tiles_dir, tile_name + ".tif")

        for positive_image in positive_images:
            y, x = os.path.splitext(positive_image)[0].split("_")[-2:] #name of tif with the extension removed; y=row;x=col
            y = int(y)
            x = int(x)
            chip = fc.tile_to_chip_array(tile, x, y, item_dim)

            #quick confirm the correct chip is selected
            gray_labeled_image = cv2.cvtColor(cv2.imread(os.path.join(positive_image_dir, positive_image)), cv2.COLOR_BGR2GRAY) #image that had been labeled
            gray_known_image = cv2.cvtColor(chip, cv2.COLOR_BGR2GRAY) #image that has been chipped from tile
            (score, diff) = compare_ssim(gray_labeled_image, gray_known_image, full=True)
            if score > 0.9: 
                chip_names.append(positive_image) # The index is a six-digit number like '000023'.
                row_indicies.append(y)
                col_indicies.append(x)
                #get the pixel coordinates
                minx = x*item_dim
                miny = y*item_dim
                maxx = x*item_dim + item_dim - 1
                maxy = y*item_dim + item_dim - 1
                NW_coordinates_pixel.append([minx, miny]) #NW (max: Top Left) # used for numpy crop
                SE_coordinates_pixel.append([maxx, maxy]) #SE (min: Bottom right) 
                #determine the lat/lon
                NW_coordinates_lonlat.append([lons[:, minx], lats[miny]]) #NW (max: Top Left) # used for numpy crop
                SE_coordinates_lonlat.append([lons[:, maxx], lats[maxy]]) #SE (min: Bottom right) 
                #tile characteristics
                standard_tile_names.append(tile_name)
                #path
                #full_path = verified_positive_jpgs[verified_positive_jpgs[:,0] == positive_image][0][1]
                #root = full_path.split("\\",2)[0]
            else:
                print("image unknown")

    #create pandas dataframe
    image_characteristics = pd.DataFrame(data={ "root":root,
                                                    'standard_tile_name': standard_tile_names,
                                                    'chip_name': chip_names,
                                                    'NW_pixel_coordinates_pixel': NW_coordinates_pixel,
                                                    'SE_pixel_coordinates_pixel': SE_coordinates_pixel,
                                                    'NW_pixel_coordinates_lonlat': NW_coordinates_lonlat,
                                                    'SE_pixel_coordinates_lonlat': SE_coordinates_lonlat,
                                                    'row_indicies': row_indicies,
                                                    'col_indicies': col_indicies})
    return(image_characteristics)

In [80]:

        fig, (ax1, ax2) = plt.subplots(1, 2)
        ax1.set_title('correct_image')
        ax1.imshow(gray_known_image)
        ax2.set_title('labeled_chip_array')
        ax2.imshow(gray_labeled_image)
        plt.show()    

'21'

In [7]:
rechipped_image_path = os.path.join(parent_directory,"temp/rechip")
image_characteristics(rechipped_image_path, tiles_complete_dataset_path)#, verified_positive_jpgs)

NameError: name 'verified_positive_jpgs' is not defined

In [33]:
dataset = rasterio.open(os.path.join(tiles_complete_dataset_path, os.listdir(tiles_complete_dataset_path)[0]))
dataset.crs

"""
#tile_height,  tile_width,  tile_channels = dataset.shape
left = dataset.bounds[0]
right = dataset.bounds[2]
bottom = dataset.bounds[1]
top = dataset.bounds[3]
x = dataset.shape[0] 
y = dataset.shape[1] 
np.linspace(left, right, num=x-1)# endpoint=True, retstep=False, dtype=None, axis=0)
"""



CRS.from_epsg(26914)

In [51]:
y0 = 0
x0 = 0
item_dim = 512

x[y0*item_dim:y0*item_dim+item_dim, x0*(item_dim):x0*(item_dim)+item_dim]

array([[-99.25351586, -99.25350991, -99.25350396, ..., -99.25048646,
        -99.25048051, -99.25047456],
       [-99.25351586, -99.25350991, -99.25350396, ..., -99.25048646,
        -99.25048051, -99.25047456],
       [-99.25351586, -99.25350991, -99.25350396, ..., -99.25048646,
        -99.25048051, -99.25047456],
       ...,
       [-99.25351586, -99.25350991, -99.25350396, ..., -99.25048646,
        -99.25048051, -99.25047456],
       [-99.25351586, -99.25350991, -99.25350396, ..., -99.25048646,
        -99.25048051, -99.25047456],
       [-99.25351586, -99.25350991, -99.25350396, ..., -99.25048646,
        -99.25048051, -99.25047456]])

In [14]:
tile.coords
SE coordinates # min 
x*item_dim, y*item_dim
NW coordinates #max
clip based on max min
#https://automating-gis-processes.github.io/CSC18/lessons/L6/clipping-raster.html

AttributeError: 'DatasetReader' object has no attribute 'coords'

In [ ]:
#specify folder that holds tiles in completed dataset

#unique positive jpgs (file names with the file extension)
unique_positive_jpgs = fc.unique_positive_jpgs_from_parent_directory(args.parent_directory)

image_characteristics = fc.image_characteristics(complete_dataset_tiles_folder_path, unique_positive_jpgs)

image_characteristics.to_csv('image_characteristics.csv')
counterin = 0
counternot = 0 

#Check to see how many images are not yet in the image characteristics folder (not verified)
for unique_jpg in unique_positive_jpgs[:,0]:
    if image_characteristics['six_digit_chip_name'].isin([unique_jpg]).any():
        counterin += 1
    if not image_characteristics['six_digit_chip_name'].isin([unique_jpg]).any():
        counternot += 1
print("images included in the image characteristics csv ",counterin, \
      "images not included in the image characteristics csv \ left to be verified", counternot)

In [ ]:
#for unique_standard_tile_names in tile_names_tile_urls_complete_array_unique_standard_tile_names[:,2]:
#    if unique
tiles_with_no_images = []
tiles_with_images = []
for standard_tile_name in tqdm_notebook(image_characteristics.standard_tile_name.unique()): #Iterate over all the possible tiles that could be included in the dataset
    print(standard_tile_name)
    #get the image characteristics for the images corresponding to each tile
    images_in_tile = image_characteristics.loc[image_characteristics.standard_tile_name == standard_tile_name] #get the annotated images corresponding to ech time
    indicies = images_in_tile.index
    
    #get the characteristics for each time
    da = xr.open_rasterio(os.path.join(tiles_complete_dataset_path, standard_tile_name +".tif"))
    tile_band, tile_height, tile_width = da.shape[0], da.shape[1], da.shape[2]
    #make the xml for each tile
    generate_xml(standard_tile_name, tiles_complete_dataset_path, tiles_xml_complete_dataset_path, tile_band, tile_height, tile_width)
    #add data from each image xml to the corresponding tile xml
    #for i in indicies:
    #    xml_path, root = get_xml_path_from_root_jpg(images_in_tile, i)
    #if not images_in_tile.empty:

In [ ]:
da.crs